In [123]:
import csv
import os
import time
import ujson
from random import randint
from typing import Dict, List, Any

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

In [133]:
def crawlDetail(driver, uri):
    driver.get(uri)
    abstract = driver.find_element(
        By.CSS_SELECTOR,"div[class='content-content publication-content']>div"
    )
    abs_text= abstract.text if abstract is not None else ""
    uri = uri + "/fingerprints/"
    driver.get(uri)
    finger_print = driver.find_elements(
        By.CSS_SELECTOR,"div[class='publication-fingerprint-thesauri']>h3"›
    )
    finger_text = [x.text for x in finger_print] if finger_print is not None else []
    return [
        abs_text,
        finger_text
    ]

In [124]:
def write_authors(list1, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for i in range(0, len(list1)):
            f.write(list1[i] + '\n')

In [134]:
pub_data = []

In [135]:
def threadRipper(links):
    global pub_data
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    for link in links:
        print(link)
        # Visit each link to get data
        result = driver.get(link)
        driver.get(link)
        try:
            name = driver.find_element(By.CSS_SELECTOR,"div[class='header person-details']>h1").text
            if driver.find_elements(By.CSS_SELECTOR,".portal_link.btn-primary.btn-large"):
                element = driver.find_elements(By.CSS_SELECTOR,".portal_link.btn-primary.btn-large")
                for a in element:
                    if "research output".lower() in a.text.lower():
                        driver.execute_script("arguments[0].click();", a)
                        driver.get(driver.current_url)
                        r = requests.get(driver.current_url)
                        soup = BeautifulSoup(r.content, 'lxml')
                        table = soup.find('ul', attrs={'class': 'list-results'})
                        if table != None:
                            for row in table.findAll('div', attrs={'class': 'result-container'}):
                                data = {}
                                data['name'] = row.h3.a.text
                                data['pub_url'] = row.h3.a['href']
                                date = row.find("span", class_="date")

                                rowitem = row.find_all(['div'])
                                span = row.find_all(['span'])
                                data['cu_author'] = name
                                data['date'] = date.text
                                
                                l3 = crawlDetail(driver, data['pub_url'])
                                data["abstract"] = l3[0]
                                data["category"] = l3[1]
                                pub_data.append(data)
            else:
                r = requests.get(link)
                soup = BeautifulSoup(r.content, 'lxml')
                table = soup.find('div', attrs={'class': 'relation-list relation-list-publications'})
                if table != None:
                    for row in table.findAll('div', attrs={'class': 'result-container'}):
                        data = {}
                        data["name"] = row.h3.a.text
                        data['pub_url'] = row.h3.a['href']
                        date = row.find("span", class_="date")
                        rowitem = row.find_all(['div'])
                        span = row.find_all(['span'])
                        data['cu_author'] = name
                        data['date'] = date.text
                        
                        l3 = crawlDetail(driver, data['pub_url'])
                        data["abstract"] = l3[0]
                        data["category"] = l3[1]
                        pub_data.append(data)
        except Exception as e:
            print(e)
            continue
    driver.quit()

In [136]:
import threading
def initCrawlerScraper(uri):
    global pub_data
    # Initialize driver for Chrome
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(uri)
    
    Links = [] 
    nextLink = driver.find_element(By.CSS_SELECTOR,".nextLink").is_enabled()
    while (nextLink):
        page = driver.page_source
        bs = BeautifulSoup(page, "lxml")

        for link in bs.findAll('a', class_='link person'):
            url = str(link)[str(link).find('https://pureportal.coventry.ac.uk/en/persons/'):].split('"')
            Links.append(url[0])
        try:
            if driver.find_element(By.CSS_SELECTOR,".nextLink"):
                element = driver.find_element(By.CSS_SELECTOR,".nextLink")
                driver.execute_script("arguments[0].click();", element)
            else:
                nextLink = False
        except NoSuchElementException:
            break
    write_authors(Links, 'Authors_URL.txt')
    Links = Links
    chunk_size = len(Links) // 25  # Calculate the size of each chunk
    chunks = [Links[i:i+chunk_size] for i in range(0, len(Links), chunk_size)]

    # If there are remaining items, add them to the last chunk
    if len(Links) % chunk_size != 0:
        chunks[-1].extend(Links[-(len(Links) % chunk_size):])
    try:
        _threader = []
        for thread in chunks:
            t = threading.Thread(target=threadRipper, args=(thread,))
            _threader.append(t)
            t.start()
        for t in _threader:
            t.join()
    except:
        pass

    print("Crawler has scrapped data for ", len(pub_data), " pureportal publications")
    driver.quit()

    with open('crawler.json', 'w') as f:
        ujson.dump(pub_data, f)

In [137]:
pub_data = []
initCrawlerScraper('https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/')

/var/folders/3y/yxfc9cms7ss262pddd2ymh_40000gp/T/ipykernel_1602/1973978386.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
/var/folders/3y/yxfc9cms7ss262pddd2ymh_40000gp/T/ipykernel_1602/138250350.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


https://pureportal.coventry.ac.uk/en/persons/shenal-rajintha-alexander-samarathunge-gunawardena
https://pureportal.coventry.ac.uk/en/persons/jane-coad
https://pureportal.coventry.ac.uk/en/persons/ikenna-stanley-enebuse
https://pureportal.coventry.ac.uk/en/persons/beth-gardiner
https://pureportal.coventry.ac.uk/en/persons/aliano-abbasi
https://pureportal.coventry.ac.uk/en/persons/linda-lee-davies
https://pureportal.coventry.ac.uk/en/persons/karolina-magdalena-mienska
https://pureportal.coventry.ac.uk/en/persons/michael-hills
https://pureportal.coventry.ac.uk/en/persons/nina-atkinson
https://pureportal.coventry.ac.uk/en/persons/rachel-bevins
https://pureportal.coventry.ac.uk/en/persons/daniel-ray
https://pureportal.coventry.ac.uk/en/persons/lamis-jamil
https://pureportal.coventry.ac.uk/en/persons/ibrahim-alkhataybeh
https://pureportal.coventry.ac.uk/en/persons/scott-delahunta
https://pureportal.coventry.ac.uk/en/persons/caroline-nalule
https://pureportal.coventry.ac.uk/en/persons/frances

https://pureportal.coventry.ac.uk/en/persons/suzanne-hilton
https://pureportal.coventry.ac.uk/en/persons/carol-pedley
https://pureportal.coventry.ac.uk/en/persons/dominic-thomas-allan
https://pureportal.coventry.ac.uk/en/persons/mehtap-hisarciklilar
https://pureportal.coventry.ac.uk/en/persons/mirabel-pelton
https://pureportal.coventry.ac.uk/en/persons/charlotte-allen
https://pureportal.coventry.ac.uk/en/persons/ami-widyastuti
Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x00000001031a78ac chromedriver + 4257964
1   chromedriver                        0x000000010319ff40 chromedriver + 4226880
2   chromedriver                        0x0000000102ddc9d4 chromedriver + 281044
3   chromedriver                        0x0000000102de9e94 chromedriver + 335508
4   chromedriver                        0x0000000102de15c8 chromedriver + 300488
5   chromedriver                        0x0

https://pureportal.coventry.ac.uk/en/persons/mohammed-abdullahi
Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x000000010105b8ac chromedriver + 4257964
1   chromedriver                        0x0000000101053f40 chromedriver + 4226880
2   chromedriver                        0x0000000100c909d4 chromedriver + 281044
3   chromedriver                        0x0000000100c9de94 chromedriver + 335508
4   chromedriver                        0x0000000100c955c8 chromedriver + 300488
5   chromedriver                        0x0000000100c9410c chromedriver + 295180
6   chromedriver                        0x0000000100c96b34 chromedriver + 305972
7   chromedriver                        0x0000000100c96c24 chromedriver + 306212
8   chromedriver                        0x0000000100cc60ec chromedriver + 499948
9   chromedriver                        0x0000000100cc11cc chromedriver + 479692
10  c

https://pureportal.coventry.ac.uk/en/persons/mick-williams
https://pureportal.coventry.ac.uk/en/persons/sam-tadrus
https://pureportal.coventry.ac.uk/en/persons/rozailin-abdul-rahman
https://pureportal.coventry.ac.uk/en/persons/golly-minkah
https://pureportal.coventry.ac.uk/en/persons/engin-erken
https://pureportal.coventry.ac.uk/en/persons/morgan-williams
https://pureportal.coventry.ac.uk/en/persons/mark-hodds
https://pureportal.coventry.ac.uk/en/persons/ainul-mardiah
https://pureportal.coventry.ac.uk/en/persons/fozia-mir
https://pureportal.coventry.ac.uk/en/persons/golnaz-taghavi-pourian-azar-2
https://pureportal.coventry.ac.uk/en/persons/julio-escudero
https://pureportal.coventry.ac.uk/en/persons/nichola-williams
https://pureportal.coventry.ac.uk/en/persons/rakib-abdur
https://pureportal.coventry.ac.uk/en/persons/azzouz-essamri
https://pureportal.coventry.ac.uk/en/persons/adriana-marin
https://pureportal.coventry.ac.uk/en/persons/behnam-mirgol
https://pureportal.coventry.ac.uk/en/per

...

https://pureportal.coventry.ac.uk/en/persons/marcus-maloney
https://pureportal.coventry.ac.uk/en/persons/nikos-fytas
Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x00000001031a78ac chromedriver + 4257964
1   chromedriver                        0x000000010319ff40 chromedriver + 4226880
2   chromedriver                        0x0000000102ddc9d4 chromedriver + 281044
3   chromedriver                        0x0000000102de9e94 chromedriver + 335508
4   chromedriver                        0x0000000102de15c8 chromedriver + 300488
5   chromedriver                        0x0000000102de010c chromedriver + 295180
6   chromedriver                        0x0000000102de2b34 chromedriver + 305972
7   chromedriver                        0x0000000102de2c24 chromedriver + 306212
8   chromedriver                        0x0000000102e120ec chromedriver + 499948
9   chromedriver                 

https://pureportal.coventry.ac.uk/en/persons/obinna-ugwu-2
https://pureportal.coventry.ac.uk/en/persons/osita-nnanna-ugwueze
https://pureportal.coventry.ac.uk/en/persons/robert-werner-de-sondberg
https://pureportal.coventry.ac.uk/en/persons/chioma-ukachukwu
https://pureportal.coventry.ac.uk/en/persons/ania-sadkowska
https://pureportal.coventry.ac.uk/en/persons/arun-ulahannan
https://pureportal.coventry.ac.uk/en/persons/sergio-mamede
https://pureportal.coventry.ac.uk/en/persons/andy-manapany
https://pureportal.coventry.ac.uk/en/persons/susanne-wessendorf
https://pureportal.coventry.ac.uk/en/persons/alessia-mangiavillano
https://pureportal.coventry.ac.uk/en/persons/kaddijatou-manneh-2
https://pureportal.coventry.ac.uk/en/persons/brandon-manning
https://pureportal.coventry.ac.uk/en/persons/mohammad-manoochehri
https://pureportal.coventry.ac.uk/en/persons/nushra-mansuri
Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0 

https://pureportal.coventry.ac.uk/en/persons/marwah-amer-kadhim-shnaiter
Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x00000001052d38ac chromedriver + 4257964
1   chromedriver                        0x00000001052cbf40 chromedriver + 4226880
2   chromedriver                        0x0000000104f089d4 chromedriver + 281044
3   chromedriver                        0x0000000104f15e94 chromedriver + 335508
4   chromedriver                        0x0000000104f0d5c8 chromedriver + 300488
5   chromedriver                        0x0000000104f0c10c chromedriver + 295180
6   chromedriver                        0x0000000104f0eb34 chromedriver + 305972
7   chromedriver                        0x0000000104f0ec24 chromedriver + 306212
8   chromedriver                        0x0000000104f3e0ec chromedriver + 499948
9   chromedriver                        0x0000000104f391cc chromedriver + 479

Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x00000001028ff8ac chromedriver + 4257964
1   chromedriver                        0x00000001028f7f40 chromedriver + 4226880
2   chromedriver                        0x00000001025349d4 chromedriver + 281044
3   chromedriver                        0x0000000102541e94 chromedriver + 335508
4   chromedriver                        0x00000001025395c8 chromedriver + 300488
5   chromedriver                        0x000000010253810c chromedriver + 295180
6   chromedriver                        0x000000010253ab34 chromedriver + 305972
7   chromedriver                        0x000000010253ac24 chromedriver + 306212
8   chromedriver                        0x000000010256a0ec chromedriver + 499948
9   chromedriver                        0x00000001025651cc chromedriver + 479692
10  chromedriver                        0x00000001025a67e4 chromedriv

https://pureportal.coventry.ac.uk/en/persons/tinashe-bvirindi
https://pureportal.coventry.ac.uk/en/persons/joe-bwembya-2
https://pureportal.coventry.ac.uk/en/persons/abdelaziz-gamil
https://pureportal.coventry.ac.uk/en/persons/eko-nur-cahyo
https://pureportal.coventry.ac.uk/en/persons/rui-cai
https://pureportal.coventry.ac.uk/en/persons/alfred-gand
Message: stale element reference: stale element not found
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x00000001046038ac chromedriver + 4257964
1   chromedriver                        0x00000001045fbf40 chromedriver + 4226880
2   chromedriver                        0x00000001042389d4 chromedriver + 281044
3   chromedriver                        0x0000000104245e94 chromedriver + 335508
4   chromedriver                        0x000000010423d5c8 chromedriver + 300488
5   chromedriver                        0x000000010423c10c chromedriver + 295180
6   chromedriver                        0x

In [125]:
with open('crawler.json', 'w') as f:
    ujson.dump(pub_data, f)

In [ ]:
import schedule
import time

schedule.every().monday.at("12:00").do(
    lambda: initCrawlerScraper("https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/")
)


while True:
    schedule.run_pending()
    time.sleep(1)

In [129]:
# def initCrawlerScraper(seed):
#     # Initialize driver for Chrome
#     webOpt = webdriver.ChromeOptions()
#     webOpt.add_experimental_option('excludeSwitches', ['enable-logging'])
#     webOpt.add_argument('--ignore-certificate-errors')
#     webOpt.add_argument('--incognito')
#     webOpt.add_argument('--headless')
    
#     driver = webdriver.Chrome(ChromeDriverManager().install(), options=webOpt)
#     driver.get(seed)  # Start with the original link
    
#     pub_data = []
#     print("Crawler has found ", len(links), " pureportal profiles")
# #     write_authors(Links, 'Authors_URL.txt')
    
    
#     print("Scraping publication data for ", len(links), " pureportal profiles...")
#     count = 0
#     for link in links:
#         # Visit each link to get data
#         time.sleep(1)
#         result = driver.get(link[0])
#         driver.get(link[0])
#         try:
#             if driver.find_elements(By.CSS_SELECTOR,".portal_link.btn-primary.btn-large"):
#                 element = driver.find_elements(By.CSS_SELECTOR,".portal_link.btn-primary.btn-large")
#                 for a in element:
#                     if "research output".lower() in a.text.lower():
#                         driver.execute_script("arguments[0].click();", a)
#                         driver.get(driver.current_url)
#                         # Get name of Author
#                         name = driver.find_element(By.CSS_SELECTOR,"div[class='header person-details']>h1")
#                         r = requests.get(driver.current_url)
#                         # Parse all the data via BeautifulSoup
#                         soup = BeautifulSoup(r.content, 'lxml')

#                         # Extracting publication name, publication url, date and CU Authors
#                         table = soup.find('ul', attrs={'class': 'list-results'})
#                         if table != None:
#                             for row in table.findAll('div', attrs={'class': 'result-container'}):
#                                 data = {}
#                                 data['name'] = row.h3.a.text
#                                 data['pub_url'] = row.h3.a['href']
#                                 date = row.find("span", class_="date")

#                                 rowitem = row.find_all(['div'])
#                                 span = row.find_all(['span'])
#                                 data['cu_author'] = name.text
#                                 data['date'] = date.text
#                                 print("Publication Name :", row.h3.a.text)
#                                 print("Publication URL :", row.h3.a['href'])
#                                 print("CU Author :", name.text)
#                                 print("Date :", date.text)
#                                 print("\n")
#                                 pub_data.append(data)
#             else:
#                 # Get name of Author
#                 name = driver.find_element(By.CSS_SELECTOR,"div[class='header person-details']>h1")
#                 r = requests.get(link)
#                 # Parse all the data via BeautifulSoup
#                 soup = BeautifulSoup(r.content, 'lxml')
#                 # Extracting publication name, publication url, date and CU Authors
#                 table = soup.find('div', attrs={'class': 'relation-list relation-list-publications'})
#                 if table != None:
#                     for row in table.findAll('div', attrs={'class': 'result-container'}):
#                         data = {}
#                         data["name"] = row.h3.a.text
#                         data['pub_url'] = row.h3.a['href']
#                         date = row.find("span", class_="date")
#                         rowitem = row.find_all(['div'])
#                         span = row.find_all(['span'])
#                         data['cu_author'] = name.text
#                         data['date'] = date.text
#                         print("Publication Name :", row.h3.a.text)
#                         print("Publication URL :", row.h3.a['href'])
#                         print("CU Author :", name.text)
#                         print("Date :", date.text)
#                         print("\n")
#                         pub_data.append(data)
#         except Exception:
#             continue

#     print("Crawler has scrapped data for ", len(pub_data), " pureportal publications")
#     driver.quit()
#     # Writing all the scraped results in a file with JSON format
#     with open('scraper_results.json', 'w') as f:
#         ujson.dump(pub_data, f)

In [130]:
# initCrawlerScraper('https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/')

/var/folders/3y/yxfc9cms7ss262pddd2ymh_40000gp/T/ipykernel_1602/1229254135.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=webOpt)


Crawler has found  546  pureportal profiles
Scraping publication data for  546  pureportal profiles...
Publication Name : Studying disruptive events: Innovations in behaviour, opportunities for lower carbon transport policy?
Publication URL : https://pureportal.coventry.ac.uk/en/publications/studying-disruptive-events-innovations-in-behaviour-opportunities
CU Author : Helen Roby
Date : Aug 2020


Publication Name : Future pathways to mainstreaming community energy
Publication URL : https://pureportal.coventry.ac.uk/en/publications/future-pathways-to-mainstreaming-community-energy
CU Author : Helen Roby
Date : 1 Dec 2019


Publication Name : Managing My Money for the Just About Managing
Publication URL : https://pureportal.coventry.ac.uk/en/publications/managing-my-money-for-the-just-about-managing
CU Author : Helen Roby
Date : 1 Sept 2019


Publication Name : Smoothing peaks and troughs: intermediary practices to promote demand side response in smart grids
Publication URL : https://pur

Publication Name : An Extended Plant Circadian Clock Model for Characterising Flowering Time under Different Light Quality Conditions
Publication URL : https://pureportal.coventry.ac.uk/en/publications/an-extended-plant-circadian-clock-model-for-characterising-flower
CU Author : Laura Roden
Date : 9 Jan 2023


Publication Name : Associations between sleep-related heart rate variability and both sleep and symptoms of depression and anxiety: A systematic review
Publication URL : https://pureportal.coventry.ac.uk/en/publications/associations-between-sleep-related-heart-rate-variability-and-bot
CU Author : Laura Roden
Date : Jan 2023


Publication Name : Cardiovascular implications and physical activity in middle-aged and older adults with a history of COVID-19 (CV COVID): a protocol for a randomised controlled trial
Publication URL : https://pureportal.coventry.ac.uk/en/publications/cardiovascular-implications-and-physical-activity-in-middle-aged-
CU Author : Laura Roden
Date : 13 May 202

Publication Name : Factors to consider when assessing diurnal variation in sports performance: the influence of chronotype and habitual training time-of-day
Publication URL : https://pureportal.coventry.ac.uk/en/publications/factors-to-consider-when-assessing-diurnal-variation-in-sports-pe
CU Author : Laura Roden
Date : Jun 2015


Publication Name : Jasmonate signalling drives time-of-day differences in susceptibility of Arabidopsis to the fungal pathogenBotrytis cinerea
Publication URL : https://pureportal.coventry.ac.uk/en/publications/jasmonate-signalling-drives-time-of-day-differences-in-susceptibi
CU Author : Laura Roden
Date : 21 Nov 2015


Publication Name : Protea hybrid 'Carnival' (P. compacta x P. neriifolia) expresses multiple copies of a LEAFY homologue involved in floral and vegetative development.
Publication URL : https://pureportal.coventry.ac.uk/en/publications/protea-hybrid-carnival-p-compacta-x-p-neriifolia-expresses-multip
CU Author : Laura Roden
Date : May 2015


P

Publication Name : (Co-)Re-Designing the Participant Information Sheet
Publication URL : https://pureportal.coventry.ac.uk/en/publications/co-re-designing-the-participant-information-sheet
CU Author : Annette Roebuck
Date : 27 Apr 2017


Publication Name : Inclusive Smartphone Interface Design in Context: co (re) designing the PIS
Publication URL : https://pureportal.coventry.ac.uk/en/publications/inclusive-smartphone-interface-design-in-context-co-re-designing-
CU Author : Annette Roebuck
Date : 2017


Publication Name : Co-Creating Smartphone Interfaces with People With Learning Disabilities to enable all users.
Publication URL : https://pureportal.coventry.ac.uk/en/publications/co-creating-smartphone-interfaces-with-people-with-learning-disab
CU Author : Annette Roebuck
Date : 6 Sept 2016


Publication Name : Co-researching smartphones. Co-creating future smartphone interfaces
Publication URL : https://pureportal.coventry.ac.uk/en/publications/co-researching-smartphones-co-creating-

...

Publication Name : Entrepreneurial Propensity of Innovation Systems
Publication URL : https://pureportal.coventry.ac.uk/en/publications/entrepreneurial-propensity-of-innovation-systems
CU Author : Esin Yoruk
Date : Jan 2012


Publication Name : Innovation Systems and Knowledge-intensive Entrepreneurship: Analytical Framework and Guidelines for Case Study Research
Publication URL : https://pureportal.coventry.ac.uk/en/publications/innovation-systems-and-knowledge-intensive-entrepreneurship-analy
CU Author : Esin Yoruk
Date : Feb 2012


Publication Name : SAPPHO revisited: factors of innovation success in knowledge-intensive enterprises in Central and Eastern Europe
Publication URL : https://pureportal.coventry.ac.uk/en/publications/sappho-revisited-factors-of-innovation-success-in-knowledge-inten-2
CU Author : Esin Yoruk
Date : 2012


Publication Name : Innovation system and knowledge-intensive entrepreneurship: United Kingdom
Publication URL : https://pureportal.coventry.ac.uk/en/publi

Publication Name : IDF2022-1042 Diabetes Specialist Intervention in General Practices in deprived and ethnically diverse areas
Publication URL : https://pureportal.coventry.ac.uk/en/publications/idf2022-1042-diabetes-specialist-intervention-in-general-practice
CU Author : Peter Zeh, PhD, MSc, BSc, PGCTSS, PGCAPHE, FHEA, RGN
Date : 15 Mar 2023


Publication Name : IDF2022-1260 Virtual case finding and diabetes specialist support for general practices in deprived areas
Publication URL : https://pureportal.coventry.ac.uk/en/publications/idf2022-1260-virtual-case-finding-and-diabetes-specialist-support
CU Author : Peter Zeh, PhD, MSc, BSc, PGCTSS, PGCAPHE, FHEA, RGN
Date : 15 Mar 2023


Publication Name : Promoting Self-care and Management for Patients With Type 2 Diabetes Based on Lifestyle Changes Under the Concept of Person-centered Care: A Review
Publication URL : https://pureportal.coventry.ac.uk/en/publications/promoting-self-care-and-management-for-patients-with-type-2-diabe
CU Auth

Publication Name : Cracking Behavior of Gd2Zr2O7/YSZ Multi-Layered Thermal Barrier Coatings Deposited by Suspension Plasma Spray
Publication URL : https://pureportal.coventry.ac.uk/en/publications/cracking-behavior-of-gd2zr2o7ysz-multi-layered-thermal-barrier-co
CU Author : Xiang Zhang
Date : Jan 2023


Publication Name : Digital Image Correlation for Measuring Full-Field Residual Stresses in Wire and Arc Additive Manufactured Components
Publication URL : https://pureportal.coventry.ac.uk/en/publications/digital-image-correlation-for-measuring-full-field-residual-stres
CU Author : Xiang Zhang
Date : Feb 2023


Publication Name : Strain controlled fatigue behaviour of a wire + arc additive manufactured Ti-6Al-4V
Publication URL : https://pureportal.coventry.ac.uk/en/publications/strain-controlled-fatigue-behaviour-of-a-wire-arc-additive-manufa
CU Author : Xiang Zhang
Date : 1 Jun 2023


Publication Name : Assessment of dynamic mode-I delamination driving force in double cantilever beam 

Publication Name : Evaluation of residual stresses induced by cold spraying of Ti-6Al-4V on Ti-6Al-4V substrates
Publication URL : https://pureportal.coventry.ac.uk/en/publications/evaluation-of-residual-stresses-induced-by-cold-spraying-of-ti-6a
CU Author : Xiang Zhang
Date : 25 Sept 2019


Publication Name : Experimental and numerical study of process-induced defects and their effect on fatigue debonding in composite joints
Publication URL : https://pureportal.coventry.ac.uk/en/publications/experimental-and-numerical-study-of-process-induced-defects-and-t
CU Author : Xiang Zhang
Date : Aug 2019


Publication Name : Fatigue behaviour of corrosion pits in X65 steel pipelines
Publication URL : https://pureportal.coventry.ac.uk/en/publications/fatigue-behaviour-of-corrosion-pits-in-x65-steel-pipelines
CU Author : Xiang Zhang
Date : Mar 2019


Publication Name : Fatigue life prediction of z-fibre pinned composite laminate under mode I loading
Publication URL : https://pureportal.coventry.

CU Author : Dingchang Zheng
Date : 1 Apr 2021


Publication Name : Extraction of Coronary Atherosclerotic Plaques From Computed Tomography Imaging: A Review of Recent Methods
Publication URL : https://pureportal.coventry.ac.uk/en/publications/extraction-of-coronary-atherosclerotic-plaques-from-computed-tomo
CU Author : Dingchang Zheng
Date : 10 Feb 2021


Publication Name : Filtering-induced time shifts in photoplethysmography pulse features measured at different body sites: the importance of filter definition and standardization
Publication URL : https://pureportal.coventry.ac.uk/en/publications/filtering-induced-time-shifts-in-photoplethysmography-pulse-featu
CU Author : Dingchang Zheng
Date : 28 Jul 2021


Publication Name : Influence of aging and increased blood pressure on oscillometric cuff pressure waveform characteristics
Publication URL : https://pureportal.coventry.ac.uk/en/publications/influence-of-aging-and-increased-blood-pressure-on-oscillometric-
CU Author : Dingchang Zh